<a id="TOC"></a>
# Table of Contents

##################################################################################
# A. [Introduction](#Intro)
# B. [Create new Blocks](#NewBlocks)
# C. [Link the blocks in to a chain](#LinkBlocks)                      				
# D. [Check if new blocks are valid (consequently the chain)](#Validity)     
# E. [The BlockChain Architecture](#Architecture)
# F. [Conclusions and Extensions](#Conclusions)
##################################################################################

<a id="Intro"></a>
# Introduction


In this notebook, I will walk you through the steps of building a very simple Blockchain network.

 A Blockchain is a distributed database with a set of rules for verifying new additions to the database. 

### Scenario

In this use case, we aim to track accounts of two people. Ali and Nadeem, who will trade digital money with each other.

We need to 

1. Create a pool of incoming transactions
2. Validate the transactions
3. Link them together to form a Block

To track our transactions, we will use a "hash function". 

This hash function will link our blocks together. 

To make it easier, we will write a helper function to wrap the python hash function.  


In [0]:
import hashlib, json, sys

def hashMe(msg=""):
    # This is a helper function that wraps our hashing algorithm
    if type(msg)!=str:
        msg = json.dumps(msg,sort_keys=True)  # If we don't sort keys, we can't guarantee repeatability!
        
    if sys.version_info.major == 2: #sys.version_info[0], A tuple containing the five components of the version number: major, minor, micro, releaselevel, and serial.
        return unicode(hashlib.sha256(msg).hexdigest(),'utf-8') #Secure Hash or Message Digest
    else:
        return hashlib.sha256(str(msg).encode('utf-8')).hexdigest()

In [0]:
import random
random.seed(0)    # Pseudo-random number generator

def makeTransaction(maxValue=3):
    # This will create valid transactions in the range of (1,maxValue)
    sign      = int(random.getrandbits(1))*2 - 1   # This will randomly choose -1 or 1
    amount    = random.randint(1,maxValue)
    aliPays = sign * amount
    nadeemPays   = -1 * aliPays
    # By construction, this will always return transactions that respect the conservation of tokens.
    # However, note that we have not done anything to check whether these overdraft an account
    return {u'Ali':aliPays,u'Nadeem':nadeemPays}

In [0]:
# Create a large set of transactions, then link them into blocks
txnBuffer = [makeTransaction() for i in range(30)]

Next step: making our very own blocks! 

#### -> We’ll take the first k transactions from the transaction buffer, and turn them into a block. 

#### -> Before we do that, we need to define a method for checking the valididty of the transactions 
   #we’ve pulled into the block.

For bitcoin, the validation function checks that the input values are
"valid unspent transaction outputs" (UTXOs), 

so that the outputs of the transaction are no greater than the input, and that the keys used for the signatures are valid. 

In Ethereum, the validation function checks that the "smart contracts" were faithfully executed and respect gas limits.

We are not going to build such a complicated system. 

We’ll define our own, very simple set of rules which make sense for a basic token system:

#### 1.  The sum of deposits and withdrawals must be 0 (tokens are neither created nor destroyed)
#### 2. A user’s account must have sufficient funds to cover any withdrawals

If either of these conditions are violated, we’ll reject the transaction.


In [0]:
def updateState(txn, state):
    # Inputs: txn, state: dictionaries keyed with account names, holding numeric values for transfer amount (txn) or account balance (state)
    # Returns: Updated state, with additional users added to state if necessary
    # NOTE: This does not not validate the transaction- just updates the state!
    
    # If the transaction is valid, then update the state
    state = state.copy() # As dictionaries are mutable, let's avoid any confusion by creating a working copy of the data.
    for key in txn:
        if key in state.keys():
            state[key] += txn[key]
        else:
            state[key] = txn[key]
    return state

What are the keys here?


In [0]:
def isValidTxn(txn,state):
    # Assume that the transaction is a dictionary keyed by account names

    # Check that the sum of the deposits and withdrawals is 0
    if sum(txn.values()) is not 0:
        return False
    
    # Check that the transaction does not cause an overdraft
    for key in txn.keys():
        if key in state.keys(): 
            acctBalance = state[key]
        else:
            acctBalance = 0
        if (acctBalance + txn[key]) < 0:
            return False
    
    return True

Let's do a dry run

In [9]:
state = {u'Ali':5,u'Nadeem':5}

print(isValidTxn({u'Ali': -3, u'Nadeem': 3},state))  # Basic transaction- this works great!
print(isValidTxn({u'Ali': -4, u'Nadeem': 3},state))  # But we can't create or destroy tokens!
print(isValidTxn({u'Ali': -6, u'Nadeem': 6},state))  # We also can't overdraft our account.
print(isValidTxn({u'Ali': -4, u'Nadeem': 2,'Lisa':2},state)) # Creating new users is valid
print(isValidTxn({u'Ali': -4, u'Nadeem': 3,'Lisa':2},state)) # But the same rules still apply!

True
False
False
True
False


Each block contains a batch of transactions, a reference to the hash of the previous block (if block number is greater than 1), and a hash of its contents and the header

[Back To The Table of Contents](#TOC)

##########################################################
<a id="NewBlocks"></a>
# Building the Blockchain: From Transactions to Blocks   
##########################################################

We’re ready to start making our blockchain! Right now, there’s nothing on the blockchain, but we can get things started by defining the ‘genesis block’ (the first block in the system).

Because the genesis block isn’t linked #to any prior block, it gets treated a bit differently, and we can arbitrarily set the system state. 

In our case, we’ll create accounts for our two users (Ali and Nadeem) and give them 50 coins each.

In [0]:
state = {u'Ali':50, u'Nadeem':50}  # Define the initial state
genesisBlockTxns = [state]
genesisBlockContents = {u'blockNumber':0,u'parentHash':None,u'txnCount':1,u'txns':genesisBlockTxns}
genesisHash = hashMe( genesisBlockContents )
genesisBlock = {u'hash':genesisHash,u'contents':genesisBlockContents}
genesisBlockStr = json.dumps(genesisBlock, sort_keys=True)

Great! This becomes the first element from which everything else will be linked.

[Back To The Table of Contents](#TOC)

<a id="LinkBlocks"></a>
# Link the Blocks to make a Chain, yes "The BlockChain"

In [0]:
chain = [genesisBlock]
#For each block, We want to collect a set of transactions, create a header, hash it, and add it to the chain

In [0]:
def makeBlock(txns,chain):
    parentBlock = chain[-1] # Why?
    parentHash  = parentBlock[u'hash']
    blockNumber = parentBlock[u'contents'][u'blockNumber'] + 1
    txnCount    = len(txns)
    blockContents = {u'blockNumber':blockNumber,u'parentHash':parentHash,
                     u'txnCount':len(txns),'txns':txns}
    blockHash = hashMe( blockContents )
    block = {u'hash':blockHash,u'contents':blockContents}
    
    return block

In [0]:
# Let’s use this to process our transaction buffer into a set of blocks:

blockSizeLimit = 5  # Arbitrary number of transactions per block- 
               #  this is chosen by the block miner, and can vary between blocks!

while len(txnBuffer) > 0:
    bufferStartSize = len(txnBuffer)
    
    ## Gather a set of valid transactions for inclusion
    txnList = []
    while (len(txnBuffer) > 0) & (len(txnList) < blockSizeLimit):
        newTxn = txnBuffer.pop()
        validTxn = isValidTxn(newTxn,state) # This will return False if txn is invalid
        
        if validTxn:           # If we got a valid state, not 'False'
            txnList.append(newTxn)
            state = updateState(newTxn,state)
        else:
            print("ignored transaction")
            sys.stdout.flush()
            continue  # This was an invalid transaction; ignore it and move on
        
    ## Make a block
    myBlock = makeBlock(txnList,chain)
    chain.append(myBlock)    

In [17]:
chain[0]

{'contents': {'blockNumber': 0,
  'parentHash': None,
  'txnCount': 1,
  'txns': [{'Ali': 50, 'Nadeem': 50}]},
 'hash': 'd22597972ec7dc520913bf0fe815c7624b7c38b0ae76a7a78d6dd44dc2ffe630'}

In [18]:
chain[1]

{'contents': {'blockNumber': 1,
  'parentHash': 'd22597972ec7dc520913bf0fe815c7624b7c38b0ae76a7a78d6dd44dc2ffe630',
  'txnCount': 5,
  'txns': [{'Ali': 1, 'Nadeem': -1},
   {'Ali': -3, 'Nadeem': 3},
   {'Ali': -2, 'Nadeem': 2},
   {'Ali': 1, 'Nadeem': -1},
   {'Ali': -2, 'Nadeem': 2}]},
 'hash': 'de67a141d0425e2b5f2ed21e4203bbf66ab19ae6e1e61326f29f0a081e95ea07'}

As expected, the genesis block includes an invalid transaction which initiates account balances (creating tokens out of thin air). The hash of the parent block is referenced in the child block, which contains a set of new #transactions which affect system state. We can now see the state of the system, updated to include the transactions:

In [19]:
state

{'Ali': 72, 'Nadeem': 28}

Confused? 

Now, let's print the whole chain to see the mah=gic?

In [20]:
chain

[{'contents': {'blockNumber': 0,
   'parentHash': None,
   'txnCount': 1,
   'txns': [{'Ali': 50, 'Nadeem': 50}]},
  'hash': 'd22597972ec7dc520913bf0fe815c7624b7c38b0ae76a7a78d6dd44dc2ffe630'},
 {'contents': {'blockNumber': 1,
   'parentHash': 'd22597972ec7dc520913bf0fe815c7624b7c38b0ae76a7a78d6dd44dc2ffe630',
   'txnCount': 5,
   'txns': [{'Ali': 1, 'Nadeem': -1},
    {'Ali': -3, 'Nadeem': 3},
    {'Ali': -2, 'Nadeem': 2},
    {'Ali': 1, 'Nadeem': -1},
    {'Ali': -2, 'Nadeem': 2}]},
  'hash': 'de67a141d0425e2b5f2ed21e4203bbf66ab19ae6e1e61326f29f0a081e95ea07'},
 {'contents': {'blockNumber': 2,
   'parentHash': 'de67a141d0425e2b5f2ed21e4203bbf66ab19ae6e1e61326f29f0a081e95ea07',
   'txnCount': 5,
   'txns': [{'Ali': 3, 'Nadeem': -3},
    {'Ali': 1, 'Nadeem': -1},
    {'Ali': -1, 'Nadeem': 1},
    {'Ali': 1, 'Nadeem': -1},
    {'Ali': 3, 'Nadeem': -3}]},
  'hash': 'd0bd31d4f609dfa72cef9a08b77dac0200fb069402ce740e882f2f5f1bd4ec2b'},
 {'contents': {'blockNumber': 3,
   'parentHash': 'd0bd3

[Back To The Table of Contents](#TOC)

##########################################################
<a id="Validity"></a>
# Checking Chain Validity 

##########################################################

Now that we know how to create new blocks and link them together into a chain, let’s define functions to check that new blocks are valid- and that the whole chain is valid.

On a blockchain network, this becomes important in two ways:

When we initially set up our node, we will download the full blockchain history. After downloading the chain, we would need to run through the blockchain to compute the state of the system. 

To protect against somebody inserting invalid transactions in the initial chain, we need to check the validity of the entire chain in this initial download.
Once our node is synced with the network (has an up-to-date copy of the blockchain and a representation of system state) it will need to check the validity of new blocks that are broadcast to the network.

We will need three functions to facilitate in this:

### 1. checkBlockHash: 
A simple helper function that makes sure that the block contents match the hash
### 2. checkBlockValidity: 
Checks the validity of a block, given its parent and the current system state. We want this to return the updated state if the block is valid, and raise an error otherwise.
### 3. checkChain:
Check the validity of the entire chain, and compute the system state beginning at the genesis block. 

This will return the system state if the chain is valid, and raise an error otherwise.


In [0]:
def checkBlockHash(block):
    # Raise an exception if the hash does not match the block contents
    expectedHash = hashMe( block['contents'] )
    if block['hash']!=expectedHash:
        raise Exception('Hash does not match contents of block %s'%
                        block['contents']['blockNumber'])
    return

In [0]:
def checkBlockValidity(block,parent,state):    
    # We want to check the following conditions:
    # - Each of the transactions are valid updates to the system state
    # - Block hash is valid for the block contents
    # - Block number increments the parent block number by 1
    # - Accurately references the parent block's hash
    parentNumber = parent['contents']['blockNumber']
    parentHash   = parent['hash']
    blockNumber  = block['contents']['blockNumber']
    
    # Check transaction validity; throw an error if an invalid transaction was found.
    for txn in block['contents']['txns']:
        if isValidTxn(txn,state):
            state = updateState(txn,state)
        else:
            raise Exception('Invalid transaction in block %s: %s'%(blockNumber,txn))

    checkBlockHash(block) # Check hash integrity; raises error if inaccurate

    if blockNumber!=(parentNumber+1):
        raise Exception('Hash does not match contents of block %s'%blockNumber)

    if block['contents']['parentHash'] != parentHash:
        raise Exception('Parent hash not accurate at block %s'%blockNumber)
    
    return state

In [0]:
def checkChain(chain):
    # Work through the chain from the genesis block (which gets special treatment), 
    #  checking that all transactions are internally valid,
    #    that the transactions do not cause an overdraft,
    #    and that the blocks are linked by their hashes.
    # This returns the state as a dictionary of accounts and balances,
    #   or returns False if an error was detected

    
    ## Data input processing: Make sure that our chain is a list of dicts
    if type(chain)==str:
        try:
            chain = json.loads(chain)
            assert( type(chain)==list)
        except:  # This is a catch-all, admittedly crude
            return False
    elif type(chain)!=list:
        return False
    
    state = {}
    ## Prime the pump by checking the genesis block
    # We want to check the following conditions:
    # - Each of the transactions are valid updates to the system state
    # - Block hash is valid for the block contents

    for txn in chain[0]['contents']['txns']:
        state = updateState(txn,state)
    checkBlockHash(chain[0])
    parent = chain[0]
    
    ## Checking subsequent blocks: These additionally need to check
    #    - the reference to the parent block's hash
    #    - the validity of the block number
    for block in chain[1:]:
        state = checkBlockValidity(block,parent,state)
        parent = block
        
    return state

In [24]:
# We can now check the validity of the state:

checkChain(chain)

{'Ali': 66, 'Nadeem': 34}

In [25]:
# And even if we are loading the chain from a text file, e.g. from backup or loading it for the first time, 
# we can check the integrity of the chain and create the current state:
chainAsText = json.dumps(chain,sort_keys=True)
checkChain(chainAsText)

{'Ali': 66, 'Nadeem': 34}

[Back To The Table of Contents](#TOC)

#############################################################
<a id="Architecture"></a>
# Putting it together: The final Blockchain Architecture

#############################################################

In an actual blockchain network, new nodes would download a copy of the blockchain and verify it (as we just did above), then announce their presence on the peer-to-peer network and start listening for transactions. Bundling transactions into a block, they then pass their proposed block on to other nodes.

We’ve seen how to verify a copy of the blockchain, and how to bundle transactions into a block. If we recieve a block from somewhere else, verifying it and adding it to our blockchain is easy.

### Let’s assume that the following code runs on Node A, which mines the block:

In [0]:
import copy
nodeBchain = copy.copy(chain)
nodeBtxns  = [makeTransaction() for i in range(5)]
newBlock   = makeBlock(nodeBtxns,nodeBchain)

In [27]:
# Now assume that the newBlock is transmitted to our node, and we want to check it and update our state if it is a valid block:

print("Blockchain on Node A is currently %s blocks long"%len(chain))

try:
    print("New Block Received; checking validity...")
    state = checkBlockValidity(newBlock,chain[-1],state) # Update the state- this will throw an error if the block is invalid!
    chain.append(newBlock)
except:
    print("Invalid block; ignoring and waiting for the next block...")

print("Blockchain on Node A is now %s blocks long"%len(chain))

Blockchain on Node A is currently 6 blocks long
New Block Received; checking validity...
Blockchain on Node A is now 7 blocks long


[Back To The Table of Contents](#TOC)

#############################################################
# Conclusions and Extensions   	
<a id="Conclusions"></a>
#############################################################

Start doing it

Extend HyperLedger (or any other distributed Ledger) and build your own BlockChain Network

[Back To The Table of Contents](#TOC)

In [0]:
#@title Default title text
